<a href="https://colab.research.google.com/github/jiangenhe/insc-486-2021-spring/blob/main/week6/Week_6_lecture_groupby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Aggregation and Group Operations

In [5]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## GroupBy Mechanics

In [6]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


### Iterating Over Groups

In [7]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
1    a  two  0.478943  0.092908
4    a  one  1.965781  1.246435
b
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
3    b  two -0.555730  0.769023


In [8]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.204708  1.393406
4    a  one  1.965781  1.246435
('a', 'two')
  key1 key2     data1     data2
1    a  two  0.478943  0.092908
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.519439  0.281746
('b', 'two')
  key1 key2    data1     data2
3    b  two -0.55573  0.769023


### Selecting a Column or Subset of Columns

In [13]:
df.groupby('key1')['data1'] 
df.groupby('key1')[['data2']]

### Grouping with Dicts and Series

In [21]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Information', 'Information', 'Communication', 'Marketing', 'Accounting'])

people

,a,b,c,d,e
Information,0.862580,-0.010032,0.050009,0.670216,0.852965
Information,-0.955869,-0.023493,-2.304234,-0.652469,-1.218302
Communication,-1.332610,1.074623,0.723642,0.690002,1.001543
Marketing,-0.503087,-0.622274,-0.921169,-0.726213,0.222896
Accounting,0.051316,-1.157719,0.816707,0.433610,1.010737


In [22]:
mapping = {'Information': 'CCI', 'Communication': 'CCI',
           'Marketing': 'Business', 'Accounting': 'Business'}

In [23]:
by_column = people.groupby(mapping)
by_column.sum()

,a,b,c,d,e
Business,-0.451771,-1.779994,-0.104462,-0.292604,1.233632
CCI,-1.425898,1.041097,-1.530583,0.707749,0.636206


### Grouping with Functions

In [25]:
people.groupby(len).sum()

,a,b,c,d,e
9,-0.503087,-0.622274,-0.921169,-0.726213,0.222896
10,0.051316,-1.157719,0.816707,0.433610,1.010737
11,-0.093289,-0.033525,-2.254225,0.017747,-0.365337
13,-1.332610,1.074623,0.723642,0.690002,1.001543


### Grouping by Index Levels

In [42]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=[['CCI', 'CCI', 'CCI', 'Business', 'Business'],
                             ['Information', 'Information', 'Communication', 'Marketing', 'Accounting']])
people.index.set_names(['college', 'department'], inplace=True)
people

a         b         c         d         e
college  department                                                     
CCI      Information    1.230769  1.288306  0.851814 -1.529181 -1.551715
         Information    0.297293  0.344791 -0.398103  0.429404 -0.285945
         Communication -2.228437  0.066559  0.489965  1.867926  2.070438
Business Marketing     -0.245383  0.762302  0.129015  0.627076 -1.062235
         Accounting    -1.499503  0.545154  0.400823 -1.946230  0.505032

In [45]:
people.groupby(level='college').count()

,a,b,c,d,e
college,,,,,
Business,2,2,2,2,2
CCI,3,3,3,3,3


## Data Aggregation

In [49]:
df

In [ ]:
grouped = df.groupby('key1')

In [54]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg([peak_to_peak])

,data1,data2
,peak_to_peak,peak_to_peak
key1,,
a,2.170488,1.300498
b,0.036292,0.487276


In [48]:
grouped.describe()

data1                      ...     data2                    
     count      mean       std  ...       50%       75%       max
key1                            ...                              
a      3.0  0.746672  1.109736  ...  1.246435  1.319920  1.393406
b      2.0 -0.537585  0.025662  ...  0.525384  0.647203  0.769023

[2 rows x 16 columns]

### Example: Random Sampling and Permutation

In [63]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [71]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [70]:
len(deck)

52

In [65]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

KD     10
JC     10
8H      8
10D    10
2C      2
dtype: int64

In [66]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_sasuit).apply(draw, n=2)

C  10C    10
   9C      9
D  5D      5
   6D      6
H  2H      2
   JH     10
S  QS     10
   8S      8
dtype: int64